<a href="https://colab.research.google.com/github/UPstartDeveloper/DS-1.1-Data-Analysis/blob/master/Notebooks/Grocery_sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Total grocery sales
*For sake of time, I will use pandas for this first, then come back and use SQL*

You are given the below tables, showing Store, Product, and Sales information for a chain of grocery stores. The columns are labeled in such a way that you should be able to interpret what each field is showing.

```
Store

store_id	location
91110	   New York
99525	   Los Angeles
37340	   Tokyo
32016	   Detroit
57507	   London


Product

product_id	product_name	price_usd
31331	      Apples	        2
34611	      Lettuce	       3
49760	      Chicken           5
26583	      Lemons	        1
20267	      Bread	         2

Sales

sale_id	product_id	store_id	date
1	       31331	     91110	02/20/2020
1	       31331	     91110	02/20/2020
2	       34611	     57507	02/20/2020
3	       26583	     37340	02/20/2020
3	       34611	     32016	02/20/2020
3	       20267	     99525	02/21/2020
4	       31331	     99525	02/21/2020
5	       49760	     99525	02/21/2020
6	       34611	     57507	02/21/2020
7	       31331	     91110	02/21/2020

```

Using the tables above, write a SQL query to return total sales (in dollars) by store location by product. If total sales are null for a given store location / product combination, set them to 0.

Your output should return the following columns:

```
store_id	product_id	total_sales
X	Y	Z
A	B	C
```

## 1: Pandas Solution

In [74]:
!pip install pandas==1.2.0  # allows us to do cross-joins using pd.DataFrame.merge

In [1]:
import numpy as np
import pandas as pd

print(pd.__version__)  # if < 1.2.0, plz implement a custom cross-join method

1.2.0


In [2]:
# make example data

stores = pd.DataFrame(
    {
        "store_id": [91110   , 99525, 37340, 32016, 57507  ],
        "location": ["New York", "Los Angeles", "Tokyo", "Detroit", "London"]
    }
)    
stores.head() 

,store_id,location
0,91110,New York
1,99525,Los Angeles
2,37340,Tokyo
3,32016,Detroit
4,57507,London


In [3]:
products = pd.DataFrame(
    {
        "product_id": [31331, 34611, 49760, 26583, 20267 ],
        "product_name": ["Apples", "Lettuce", "Chicken", "Lemons", "Bread"],
        "price_usd": [2, 3, 5, 1, 2]
    }
)

products.head()

,product_id,product_name,price_usd
0,31331,Apples,2
1,34611,Lettuce,3
2,49760,Chicken,5
3,26583,Lemons,1
4,20267,Bread,2


In [4]:
# for sake of time, I will not be converting the dates to actual dt values, and it shouldn't make too much of a difference in the solution
data_values = [
(1,           31331,         91110,    2/20/2020),
(1 ,          31331 ,        91110 ,   2/20/2020),
(2  ,         34611  ,       57507  ,  2/20/2020),
(3   ,        26583   ,      37340   , 2/20/2020),
(3    ,       34611    ,     32016    ,2/20/2020),
(3     ,      20267     ,    99525    ,2/21/2020),
(4      ,     31331      ,   99525    ,2/21/2020),
(5       ,    49760       ,  99525    ,2/21/2020),
(6        ,   34611        , 57507    ,2/21/2020),
(7         ,  31331         ,91110    ,2/21/2020),
]
sales = pd.DataFrame(data=np.array(data_values),
                     columns=["sale_id",    "product_id", "store_id",  "date"])

sales.head()


,sale_id,product_id,store_id,date
0,1.0,31331.0,91110.0,0.00005
1,1.0,31331.0,91110.0,0.00005
2,2.0,34611.0,57507.0,0.00005
3,3.0,26583.0,37340.0,0.00005
4,3.0,34611.0,32016.0,0.00005


In [72]:
def total_grocery_sales(stores, products, sales):
  """
  Shows the total sale amount for each store, for each product.

  Args:
  stores(pd.DataFrame): lists the unique store locations and their ids
  products(pd.DataFrame): lists the unique grocery items sold across the chain,
                          as well as their prices
  sales(pd.DataFrame): records each and every sale at every location 
                       (sorted by date), showing the product and location.
                       Quantity of the sale is shown by rows having duplicate 
                       sale_id's (int).

  Returns: pd.DataFrame

  """
  # A: Cross join the stores by each product
  total_sales = stores.merge(products, how="cross")
  # cannot select rows while cross joining, so select them here
  total_sales = total_sales[["store_id", "product_id"]]
  # B: Count the appearances of each unique store-product combo in the sales df
  sales = sales[["product_id", "store_id"]]
  store_product_counts = sales.value_counts()
  # C: init the total sales column to be 0 for all rows at first
  num_rows = total_sales.shape[0]
  total_sales["total_sales"] = pd.Series([0 for _ in range(num_rows)])
  # calculate the total sale amt = count x product unit price, for each combo
  for row_index, combination in enumerate(store_product_counts.index):
    # get values for the unique product and store IDs
    product_id, store_id = combination
    product_id, store_id = int(product_id), int(store_id)
    # find the total sale amount
    count = store_product_counts.iloc[row_index]
    unit_price = (
        products[products["product_id"] == product_id]["price_usd"].values[0]
    )
    total_sale_amt = unit_price * count
    # place it in the output df
    total_sales.loc[
      # this query first finds the row based on the product and store cols,
      (total_sales["product_id"] == product_id) & 
      (total_sales["store_id"] == store_id),
      # then it chooses the col of this row in which to set to new value 
      "total_sales"] = total_sale_amt
  # D: return the new df
  return total_sales




In [73]:
total_grocery_sales(stores, products, sales)

,store_id,product_id,total_sales
0,91110,31331,6
1,91110,34611,0
2,91110,49760,0
3,91110,26583,0
4,91110,20267,0
5,99525,31331,2
6,99525,34611,0
7,99525,49760,5
8,99525,26583,0
9,99525,20267,2


## 2. SQL Solution
TBD